In [ ]:
import autogen
import os, re
from autogen.agentchat.contrib.web_surfer import WebSurferAgent

os.environ["OPENAI_API_KEY"] = ""
os.environ["GEMINI_API_KEY"] = ""


In [3]:
config_list_gemini = [
    {
        "model": "gemini-1.5-flash",
        "api_key": os.environ.get("GEMINI_API_KEY"),
        "api_type": "google"
    }
]
config_list_gpt = [
    {
        "model": "gpt-3.5-turbo-0125",
        "api_key":os.environ.get("OPENAI_API_KEY"),
    }
]

MAX_USER_REPLIES = 5

In [ ]:
web_surfer = WebSurferAgent(
  name = "web_surfer",
  system_message="Surf the Web for the required information, always skip to main so that the relevant information can be found. Summarize the website when needed.",
  llm_config = {
        "cache_seed": 41,
        "config_list": config_list_gpt,
        "seed": 42
    },
  summarizer_llm_config = {
        "cache_seed": None,
        "config_list": config_list_gpt,
        "seed": 42
    },

  browser_config = {"viewport_size" : 4096, "bing_api_key":""}
)

second_web_surfer = WebSurferAgent(
  name = "second_web_surfer",
  system_message="You follow after a web search, go into the required website and summarize it.",
  llm_config = {
        "cache_seed": 41,
        "config_list": config_list_gpt,
        "seed": 42
    },
  summarizer_llm_config = {
        "cache_seed": None,
        "config_list": config_list_gpt,
        "seed": 42
    },

  browser_config = {"viewport_size" : 4096, "bing_api_key":"52272f9444264614ac19f70e4f45c892"}
)


coder = autogen.AssistantAgent(
    name="coder",
    llm_config={
        "cache_seed": 41,
        "config_list": config_list_gpt,
        "seed": 42
    },
    system_message="""Coder. You follow an approved plan. You write python code to solve tasks. Wrap the code in a code block that specifies the script type. Always add the filename in the format #filename:name in the code block after specifying the coding language. The user can't modify your code. So do not suggest incomplete code which requires others to modify. Don't use a code block if it's not intended to be executed by the executor.
    Don't include multiple code blocks in one response. Do not ask others to copy and paste the result. Check the execution result returned by the executor.
    If the result indicates there is an error, fix the error and output the code again. Suggest the full code instead of partial code or code changes. If the error can't be fixed or if the task is not solved even after the code is executed successfully, analyze the problem, revisit your assumption, collect additional info you need, and think of a different approach to try.
    """
)
second_coder = autogen.AssistantAgent(
    name="second_coder",
    llm_config={
        "cache_seed": 41,
        "config_list": config_list_gpt,
        "seed": 42
    },
    system_message="""Second Coder. You always output code, you get called when the first coder does not ouput code. 
    Always add the filename in the format #filename:name in the code block after specifying the coding language. 
    Output exactly what the previous coder tries to implement. 
    """
)
code_fixer = autogen.AssistantAgent(
    name="code_fixer",
    llm_config={
        "cache_seed": 41,
        "config_list": config_list_gpt,
        "seed": 42
    },
    system_message="""Code Fixer. You fix the code based on the error message and the code that was attempted to run. Always add the filename in the format #filename:name in the code block after specifying the coding language.
    Always reply will the full version of the code, the user is not able to change anything himself.
    """
)

planner = autogen.AssistantAgent(
    name="planner",
    llm_config={
        "cache_seed": 41,
        "config_list": config_list_gpt,
        "seed": 42
    },
    system_message="""
    Plan the steps needed to complete the task at hand. Use 3 or 4 steps, the plan must be possible to execute without human input.
    All the coding environments are already set up.
    Write an outlined plan and write either coder, web_surfer or assistant after each step depending on what is needed.
    If the step requires surfing the web, write web_surfer,
    If the step requires writing code, write coder,
    If the step requires general assistance, write assistant.
    """
)
assistant = autogen.AssistantAgent(
    name="assistant",
    llm_config={
        "cache_seed": 41,
        "config_list": config_list_gpt,
        "seed": 42,
        },
   
)
plan_checker = autogen.AssistantAgent(
    name="Plan_checker",
    system_message="Check the Plan of the previous Agent and decide wether we should proceed with the plan.",
    llm_config={
        "cache_seed": 41,
        "config_list": config_list_gpt,
        "seed": 42
    },
)



user_proxy = autogen.UserProxyAgent(
  name="user_proxy",
  human_input_mode="NEVER",
  max_consecutive_auto_reply=MAX_USER_REPLIES,
  is_termination_msg=lambda x: x.get("content", "").rstrip().endswith("TERMINATE"),
  code_execution_config={
        "work_dir": "coding",
        "use_docker": False
    }
)



In [ ]:
def custom_speaker_selection_func(last_speaker : autogen.agentchat.agent.Agent, groupchat):
    messages = groupchat.messages
    agents = groupchat.agents
    n = len(messages)
    for mess in messages:
        if mess["name"] == "user_proxy" or mess["name"]=="second_coder" or mess["name"]=="code_fixer" or mess["name"]=="second_web_surfer":
            n -= 1
    if "```python" in messages[-1]["content"] or "``` bash" in messages[-1]["content"]:
        return user_proxy
    if "execution failed" in messages[-1]["content"] or "[]" in messages[-1]["content"]:
        return code_fixer
    if n == 0:
        return planner
    if last_speaker.name == "coder":
        if not "```python" in messages[-1]["content"]:
            return second_coder
    if last_speaker.name == "web_surfer":
       return second_web_surfer
    else:
        messages = messages[1]["content"]
        list_of_messages = re.split("\d\.", messages)
        list_of_agents = [k[-14:] for k in list_of_messages]
        if n< len(list_of_agents):
            for i in agents:
                if i.name in list_of_agents[n]:
                    return i
        return assistant



In [ ]:
task ="""

"""

In [ ]:
groupchat = autogen.GroupChat([plan_checker, second_coder, code_fixer, web_surfer, second_web_surfer, coder,planner,assistant, user_proxy], messages=[], max_round=15, speaker_selection_method=custom_speaker_selection_func)
manager = autogen.GroupChatManager(groupchat=groupchat, name="chat_manager", llm_config= config_list_gpt[0])

In [38]:
messages = user_proxy.initiate_chat(manager, message=task)

user_proxy (to chat_manager):


Find the link to the financial statement of tesla, read it as a csv file onto my computer and then perform a cash flow analysis with this data.


--------------------------------------------------------------------------------

Next speaker: planner

planner (to chat_manager):

Here is the plan to accomplish your task:

1. Search for the official website of Tesla to find the link to the financial statement. Use a search engine to find the link. - web_surfer

2. Download the financial statement as a CSV file from the link found in step 1. - web_surfer

3. Read the CSV file into a programming environment using Python and perform the cash flow analysis with the data. - coder

--------------------------------------------------------------------------------

Next speaker: web_surfer


>>>>>>>> USING AUTO REPLY...

>>>>>>>> EXECUTING FUNCTION informational_web_search...
Call ID: call_2IqEyYUnDu9iXQWgYB6WtqIQ
Input arguments: {'query': 'Tesla official website'}